In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
sn.set()
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [3]:
seg_df=pd.read_csv("segmentation data.csv")
segment_df=seg_df.drop("ID",axis=1)
segment_df.head()

,Sex,Marital status,Age,Education,Income,Occupation,Settlement size
0,0,0,67,2,124670,1,2
1,1,1,22,1,150773,1,2
2,0,0,49,1,89210,0,0
3,0,0,45,1,171565,1,1
4,0,0,53,1,149031,1,1


In [4]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [5]:
x_scaled=scaler.fit_transform(segment_df)
x_scaled.shape

(2000, 7)

In [6]:
from sklearn.decomposition import PCA
pca=PCA(n_components=0.80,random_state=64)

In [7]:
pca_df=pd.DataFrame(pca.fit_transform(x_scaled))
pca_df

,0,1,2
0,2.514746,0.834122,2.174806
1,0.344935,0.598146,-2.211603
2,-0.651063,-0.680093,2.280419
3,1.714316,-0.579927,0.730731
4,1.626745,-0.440496,1.244909
...,...,...,...
1995,-0.866034,0.298330,1.438958
1996,-1.114957,0.794727,-1.079871
1997,-1.452298,-2.235937,0.896571
1998,-2.241453,0.627108,-0.530456


In [13]:
from sklearn.cluster import KMeans
model_kmeans=KMeans(n_clusters=4,random_state=42,max_iter=1000, tol=0.0001)
k_means_cluster=model_kmeans.fit_predict(pca_df)

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder,StandardScaler

In [26]:
column_transformer=Pipeline(
        steps=[
            ('imputer',SimpleImputer(strategy="median")),
            ('scaler',StandardScaler()),
            ('pca',PCA(n_components=0.80,random_state=64))
        ])


In [27]:
X=segment_df

In [28]:
# Unsupervised clustering pipeline
clustering_pipeline = Pipeline([
    ('kmeans', KMeans(n_clusters=4,random_state=42,max_iter=1000, tol=0.0001))
])

In [33]:
# Fit and transform the data
ext=pd.DataFrame(column_transformer.fit_transform(X))

In [34]:
new_clutsers=clustering_pipeline.fit_predict(ext)

In [35]:
sum(k_means_cluster==new_clutsers)

2000

In [37]:
maps={0:"Standard",1:"Platinum",2:"Gold",3:"Bronze"}

In [38]:
model_kmeans= KMeans(n_clusters=4,random_state=42,max_iter=1000, tol=0.0001)

In [39]:
model_kmeans.fit(ext)

KMeans(max_iter=1000, n_clusters=4, random_state=42)

In [40]:
import pickle
# Scaler 
pickle.dump(column_transformer, open('column_transformer.pickle', 'wb'))
# KMeans PCA 
pickle.dump(model_kmeans, open('model_kmeans.pickle', 'wb'))

In [41]:
model_features=['Sex', 'Marital status', 'Age', 'Education', 'Income', 'Occupation', 'Settlement size']